In [1]:
import os
import pandas as pd
import ipywidgets as widgets
import datetime
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from webdriver_manager.utils import ChromeType
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.firefox.service import Service as FirefoxService
from selenium.webdriver.chrome.options import Options as ChromeOptions
from selenium.webdriver.firefox.options import Options as FirefoxOptions
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

os.environ['WDM_LOG_LEVEL'] = '0'

In [2]:
def inicializarDriver(name):
    if name == ("google-chrome.desktop" or "brave-browser.desktop" or "chromium_chromium.desktop"):
        options = ChromeOptions()
        options.add_argument("--headless")

        if name == "google-chrome.desktop":
            driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)
        elif name == "brave-browser.desktop":
            driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager(chrome_type=ChromeType.BRAVE).install()), options=options)
        elif name == "chromium_chromium.desktop":
            driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager(chrome_type=ChromeType.CHROMIUM).install()), options=options)

    elif (name == "firefox.desktop"):
        options = FirefoxOptions()
        options.add_argument("--headless")
        driver = webdriver.Firefox(service=FirefoxService(GeckoDriverManager().install(), log_path=os.devnull), options=options)

    return driver

In [4]:
browser_name = ["google-chrome.desktop", "brave-browser.desktop", "chromium_chromium.desktop"]

i = 0
while (i < 3):
    try:
        driver = inicializarDriver(browser_name[i])
        i = 3
    except:
        i += 1

driver.set_page_load_timeout(100)

max_anos = 10
url = "http://www3.saude.ba.gov.br/cgi/deftohtm.exe?sinan/deng.def"

driver.get(url)

WebDriverWait(driver, 1).until(EC.presence_of_element_located((By.CLASS_NAME, 'mostra')))

# Linha
driver.find_element(By.XPATH, '/html/body/div/center/div/form/div[2]/div/div[1]/select/option[1]').click()
# Coluna
driver.find_element(By.XPATH, '/html/body/div/center/div/form/div[2]/div/div[2]/select/option[21]').click()
# Periodo
driver.find_element(By.XPATH, '/html/body/div/center/div/form/div[3]/div/select/option[1]').click()
for i in range (2, max_anos+3):
    driver.find_element(By.XPATH, f'/html/body/div/center/div/form/div[3]/div/select/option[{i}]').click()

# Mostra
driver.find_element(By.XPATH, '/html/body/div/center/div/form/div[4]/div[2]/div[2]/input[1]').click()

html = driver.page_source

driver.quit()

In [5]:
soup = BeautifulSoup(html, 'html.parser')

tabdados = soup.select(".tabdados tbody tr td ")

tabdados = list(map(lambda node: node.get_text().strip(), tabdados))

col_tabdados = soup.select(".tabdados th ")
col_tabdados = list(map(lambda node: node.get_text().strip(), col_tabdados))

In [6]:
tam_lin = int(len(tabdados)/len(col_tabdados))
tam_col = len(col_tabdados)

aux = []
for i in range (tam_lin):
    aux_2 = []
    for j in range (tam_col):  
        aux_2.append(tabdados.pop(0))
    aux.append(aux_2)

df = pd.DataFrame(aux, columns=col_tabdados)

df = df.drop(columns=['Total'])
df = df.drop(0)
df = df.reset_index()
df = df.drop(columns=['index'])

In [7]:
aux = []
for item in df.iloc[:, 0]:
    aux.append(item.split(" ", 1))

aux = pd.DataFrame(aux)

df["Código"] = aux[0]
df["Município"] = aux[1]

aux = list(df.columns)
aux.insert(0, aux.pop())
df = df[aux]

In [8]:
df.iloc[:, 2:] = df.iloc[:, 2:].astype(str).replace('\.', '', regex=True)
df.iloc[:, 2:] = df.iloc[:, 2:].astype(str).replace('-', '0')

In [10]:
aux = datetime.date.today()
aux = aux.strftime("%Y")

try:
    df = df.drop(columns=[aux])
except :
    pass

In [11]:
df.head()

,Código,Município,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,290030,Acajutiba,0,0,0,0,0,0,0,0,0,1,0
1,290070,Alagoinhas,0,0,0,0,0,0,0,0,0,2,2
2,290090,Almadina,0,0,0,0,1,0,0,0,0,0,0
3,290100,Amargosa,0,0,1,0,0,0,0,0,1,0,0
4,290110,Amélia Rodrigues,0,0,0,0,1,0,0,0,0,0,0


In [38]:
df.to_csv(f"../scr/dengue/obitos_municipio_{max_anos}.csv", encoding='utf-8')